### Log C V4 Content Preparation

In [2]:
import sys
#sys.path.append('../shared/')
#import videodisko as vd
# Imports
import numpy as np
import numpy.matlib
import numpy.random

from mpl_toolkits.mplot3d import axes3d 

from matplotlib import pyplot as plt

from scipy import ndimage
from scipy import misc
from scipy import fftpack
from scipy import optimize
import scipy.io
import skimage
import imageio
import colour as c
import math
import csv

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import pandas as pd

import cv2
import time

from datetime import datetime

import os
from fnmatch import fnmatch

In [3]:

## Linear to sRGB
def lin2srgb (x):
    result = (x <= 0.0031308) * np.multiply(x,12.92) + (x > 0.0031308) * np.subtract(np.multiply(1.055,np.power(x,(1/2.4))),0.055)
    return result


## sRGB to Linear 
def srgb2lin (x):
    result = (x <= 0.04045) * np.divide(x,12.92) + (x > 0.04045) * np.power(np.divide(np.add(x,0.055),1.055),2.4)
    return result

# SDR Formulas

## Input Luminance between 0-1
## Output Videosignal
def oetf709(x): 
    return np.subtract(np.multiply(np.power(x, 0.45), 1.099), 0.099)*(0.018<= x) + np.multiply(4.5, x) * (x <0.018)

def eotf1886(x, Lw = 100, Lb = 0): 
    gamma = 2.4
    a = np.power(np.subtract(np.power(Lw, np.divide(1,gamma)),np.power(Lb, np.divide(1, gamma))), gamma)
    b = np.divide(np.power(Lb, np.divide(1,gamma)) , np.subtract(np.power(Lw , np.divide(1,gamma)), np.power( Lb , np.divide(1, gamma))))
    return np.multiply(a , np.power( np.maximum(np.add(x, b), 0) , gamma))

def eotf1886_CRT(x, Lw = 100, Lb = 0, b = 0): 
    Vc = 0.35
    a1 = 2.6
    a2 = 3.0
    k = np.subtract(Lw,np.power(np.add(1,b),a1))
    return (x < Vc) * np.multiply(np.multiply(k,np.power(np.add(Vc,b),np.subtract(a1,a2))),np.power(np.add(x,b),a2)) + (Vc <= x) * np.multiply(k,np.power(np.add(x,b),a1))

## Luma Formulas for Rec2020 / Rec709 -> Fixed Addition
def luma709(img): 
    return np.multiply(img[:,:,0],0.2126) + np.multiply(img[:,:,1],0.7152) + np.multiply(img[:,:,2],0.0722)
def luma2020(img): 
    return np.multiply(img[:,:,0],0.2627) + np.multiply(img[:,:,1],0.6780) + np.multiply(img[:,:,2],0.0593)

## Formulas for PQ

# increasing rec.709 formulas 
def steroids709(x): 
    return np.subtract(np.multiply(np.power(np.multiply(59.5208 , x), 0.45), 1.099), 0.099)*(0.0003024< x) + np.multiply(267.84, x) * (x <= 0.0003024)

def steroids1886(x): 
    return np.multiply(100, np.power(x, 2.4))


## Transportfunctions

def pq_eotf(x): 
    ##m1
    m1 = np.divide(2610, 16384) ## 2^14 = 16384
    m2 = np.multiply(np.divide(2523, 4096), 128) ##2^12 = 4096 , 2^7 = 128 -> Jan fragen
    c1 = np.divide(3424, 4096)
    c2 = np.multiply(np.divide(2413, 4096), 32)
    c3 = np.multiply(np.divide(2392, 4096), 32)

    
    return np.multiply(np.power(np.divide(np.maximum( np.subtract( np.power(x , np.divide(1 , m2)), c1) , 0) , np.subtract(c2, np.multiply(c3 , np.power(x , np.divide(1 , m2))))) , np.divide(1, m1)), 10000)

## Inverse_eotf

def pq_eotf_inv(x): 
    m1 = np.divide(2610, 16384) ## 2^14 = 16384
    m2 = np.multiply(np.divide(2523, 4096), 128) ##2^12 = 4096 , 2^7 = 128 -> Jan fragen
    c1 = np.divide(3424, 4096)
    c2 = np.multiply(np.divide(2413, 4096), 32)
    c3 = np.multiply(np.divide(2392, 4096), 32)

    y = np.divide(x , 10000) ## Normalisierung auf 0.00 bis 1.0

    return np.power( np.divide(np.add(c1 , np.multiply(c2 , np.power(y , m1))), np.add(1 , np.multiply(c3 , np.power(y , m1)))), m2)

# RGB Input
def hlg_oetf(x): 
    a = 0.17883277
    b = np.subtract(1 , np.multiply(4 , a))
    c = np.subtract(0.5 , np.multiply(a , np.log(np.multiply(4 , a))))

    return np.where(x <= np.divide(1,12), np.sqrt(np.multiply(3,x )), np.add( np.multiply(a , np.log(np.subtract( np.multiply(12 , x ), b))) , c) )

def hlg_oetf_inv(x):
    a = 0.17883277
    b = np.subtract(1 , np.multiply(4 , a))
    c = np.subtract(0.5 , np.multiply(a , np.log(np.multiply(4 , a))))
    
    return np.where(x <= 0.5 , np.divide( np.power(x,2), 3), np.divide(np.add(np.exp(np.divide(np.subtract(x , c), a)), b) , 12) )

## Lw = Usergain, nominal peak Luminance of the display (achromatic)
def hlg_ootf(x, Lw): 
    r = x[: , : , 0]
    g = x[: , : , 1]
    b = x[: , : , 2]
    luma = np.add(np.multiply(0.2627 ,r ), np.multiply(0.678 , g), np.multiply(0.0593 , b))
    alpha = Lw
    gamma = np.add(1.2, np.multiply(0.42 , np.log10(np.divide(Lw,1000))))
    rd = np.multiply( alpha , np.power(luma , np.subtract(gamma , 1)), r)
    gd = np.multiply( alpha , np.power(luma , np.subtract(gamma , 1)), g)
    bd = np.multiply( alpha , np.power(luma , np.subtract(gamma , 1)), b)
    return np.stack((rd , gd , bd))

## Lw = Usergain, nominal peak Luminance of the display (achromatic)
def hlg_ootf_bw(x, Lw=1000): 
    alpha = Lw
    gamma = np.add(1.2, np.multiply(0.42 , np.log10(np.divide(Lw,1000))))
    return np.multiply(np.multiply( alpha , np.power(x , np.subtract(gamma , 1))),x)

def hlg_eotf(x, Lw = 1000, Lb = 0.005):
    gamma = np.add(1.2, np.multiply(0.42 , np.log10(np.divide(Lw,1000))))
    beta = np.sqrt(np.multiply(3 , np.power(np.divide(Lb , Lw), np.divide(1,gamma))))
    return hlg_ootf_bw(hlg_oetf_inv(np.maximum(0 , (np.add( np.multiply( np.subtract(1 , beta), x ), beta)))), Lw)


def slog3oetf(x): 
    # reflection add
    x = x*0.9

    return (x < 0.01125)* np.divide( np.add( np.multiply(x , np.divide(np.subtract(171.2102946929, 95), 0.01125)), 95), 1023) + (x >= 0.01125)* np.divide(np.add(420 , np.multiply( np.log10(np.divide(np.add(x,0.01), np.add(0.18, 0.01))), 261.5)), 1023)

def slog3eotf(x): 
    return (x >= np.divide(171.2102946929,1023)) * np.subtract( np.multiply(np.power(10 , np.divide( np.subtract(420 , np.multiply(x , 1023)) , 261.5)) , np.add(0.18 , 0.01)) , 0.01) + (x < np.divide(171.2102946929,1023)) * np.multiply(np.subtract(np.multiply(x , 1023), 95 ) , np.divide(0.01125 , np.subtract(171.2102946929 , 95)))

# These are the ARRI LogCv3 Formulas
def lin2logc(x):
    return np.multiply(x > 0.010591, 0.24719 * np.log10(5.555556 * x + 0.052272) + 0.385537) + np.multiply(x <= 0.010591, 5.367655 * x + 0.092809)
def logc2lin(x):
    return np.multiply(x > 0.149659, (np.power(10.0, (x - 0.385537) / 0.247189) - 0.052272) / 5.555556) + np.multiply(x <= 0.149659, (x - 0.092809) / 5.367650)


### LOG C4
def Encoding_LogC4(x):
    a = (np.power(2.0, 18.0) - 16.0) / 117.45
    b = (1023.0 - 95.0) / 1023.0
    c = 95.0 / 1023.0
    s = (7 * np.log(2) * np.power(2.0, 7 - 14 * c / b)) / (a * b)
    t = (np.power(2.0, 14.0 * (-c / b) + 6.0) - 64.0) / a
    return (x < t) * np.divide(np.subtract(x,t),s) + (x >= t) * np.add(np.multiply(np.divide( np.subtract(np.log2(np.add(np.multiply(a,x),64.0)),6.0),14.0),b),c)

def Decoding_LogC4(x):
    a = (np.power(2.0, 18.0) - 16.0) / 117.45
    b = (1023.0 - 95.0) / 1023.0
    c = 95.0 / 1023.0
    s = (7 * np.log(2) * np.power(2.0, 7 - 14 * c / b)) / (a * b)
    t = (np.power(2.0, 14.0 * (-c / b) + 6.0) - 64.0) / a
    return (x < 0.0) * np.add(np.multiply(x,s),t) + (x >= 0.0) * np.divide(np.subtract(np.power(2.0, np.add(np.multiply(14,np.divide(np.subtract(x,c),b)),6)),64.0),a)


def full2legal(x):
    return (219*(x)+16)*np.power(2,(10-8))

def legal2full(x):
    return (((x/np.power(2,(10-8)))-16)/219)

def lin2log2(x):
    return np.log2(x)

def log22lin(x):
    return np.power(2,x)

def Blenden_Differenz(num1, num2):
    new2 = num2 / num1
    return np.log2(new2)

def mse_calculation(imageA, imageB):
    # Expects Luma Images
	err = np.mean(np.square(np.subtract(imageA,imageB)))
	return err

In [4]:
Matrix_LMS_XYZ = [[1.91020,-1.11212,0.20191],
                  [0.37095,0.62905,0.0],
                  [0.0,0.0,1.0]]

Matrix_BT709_XYZ = [[0.4124564, 0.3575761, 0.1804375],
                    [0.2126729, 0.7151522, 0.0721750],
                    [0.0193339, 0.1191920, 0.9503041]]

Matrix_BT2020_XYZ = [[0.636958, 0.144617, 0.168881],
                    [0.262700, 0.677998, 0.059302],
                    [0.00000, 0.028073, 1.060985]]

Matrix_BT709_BT2020 = [[0.6274, 0.3293, 0.0433],
                        [0.0691, 0.9195, 0.0114],
                        [0.0164, 0.0880, 0.8956]]

Matrix_LogC4_XYZ = [[0.704858320407232064, 0.129760295170463003, 0.115837311473976537],
                        [0.254524176404027025, 0.781477732712002049, -0.036001909116029039],
                        [0.000000000000000000, 0.000000000000000000, 1.089057750759878429]]

Matrix_XYZ_BT2020 = [[1.716651, -0.355671, -0.253366],
                        [-0.666684, 1.616481, 0.015769],
                        [0.017640, -0.042771, 0.942103]]

### With Scaling to UHD-1

In [7]:
# LogC4 to PQ/HLG Conversion with Color Gamut Mapping (Conversion Matrix + Clamping)

root = '/Users/justus/Library/CloudStorage/OneDrive-Persönlich/Masterarbeit/Analysis_Content_Encoding/XX_WATCHFOLDER_LOGC4_2_PQ_HLG'
pattern = "*.tif"

for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            path = os.path.join(path)
            # Folder Settings
            new_path_hlg_FR = path.replace('XX_WATCHFOLDER_LOGC4_2_PQ_HLG',"11.1_TRANSFORMED_HLG_FR")
            new_path_hlg_NR = path.replace('XX_WATCHFOLDER_LOGC4_2_PQ_HLG',"11.2_TRANSFORMED_HLG_NR")
            new_path_pq_FR = path.replace('XX_WATCHFOLDER_LOGC4_2_PQ_HLG',"12.1_TRANSFORMED_PQ_FR_")
            new_path_pq_NR = path.replace('XX_WATCHFOLDER_LOGC4_2_PQ_HLG',"12.2_TRANSFORMED_PQ_NR_")
            # File Settings
            new_name_hlg_FR = os.path.join(new_path_hlg_FR,"TRANSFORMED_HLG_BT2020_FR_" + name)
            new_name_hlg_NR = os.path.join(new_path_hlg_NR,"TRANSFORMED_HLG_BT2020_NR_" + name)
            new_name_pq_FR = os.path.join(new_path_pq_FR,"TRANSFORMED_PQ_BT2020_FR_" + name)
            new_name_pq_NR = os.path.join(new_path_pq_NR,"TRANSFORMED_PQ_BT2020_NR_" + name)
            # make directories
            if not os.path.exists(new_path_hlg_FR):
                os.makedirs(new_path_hlg_FR)
            if not os.path.exists(new_path_hlg_NR):
                os.makedirs(new_path_hlg_NR)
            if not os.path.exists(new_path_pq_FR):
                os.makedirs(new_path_pq_FR)
            if not os.path.exists(new_path_pq_NR):
                os.makedirs(new_path_pq_NR)
            print(os.path.join(path,name))

            LogC4_image = c.read_image(os.path.join(path,name))
            height = len(LogC4_image[0,:,0])
            width = len(LogC4_image[:,0,0])

            target_height = 3840
            target_width = 2160

            LogC4_Linear = Decoding_LogC4(LogC4_image)
            Linear_XYZ = np.matmul(LogC4_Linear,Matrix_LogC4_XYZ)
            Linear_BT2020 = np.matmul(Linear_XYZ,Matrix_XYZ_BT2020)

            # Scaling Scene Linear Reflectance @ 203 Nits HDR Diffuse White
            Scaled_Linear_BT2020_HLG = Linear_BT2020 * c.models.oetf_inverse_HLG_BT2100(c.models.eotf_inverse_HLG_BT2100(203,0,1000))
            Scaled_Linear_BT2020_PQ = Linear_BT2020 * c.models.oetf_inverse_PQ_BT2100(c.models.eotf_inverse_ST2084(203))

            # Converting to Relative Video Level
            BT2020_HLG_from_LogC4 = c.models.oetf_HLG_BT2100(Scaled_Linear_BT2020_HLG)
            BT2020_PQ_from_LogC4 = c.models.oetf_PQ_BT2100(Scaled_Linear_BT2020_PQ)

            # Quantization FR
            BT2020_HLG_from_LogC4_10bit_FR = np.round(np.multiply(BT2020_HLG_from_LogC4,1023),0)
            BT2020_PQ_from_LogC4_10bit_FR = np.round(np.multiply(BT2020_PQ_from_LogC4,1023),0)

            # Quantization NR
            BT2020_HLG_from_LogC4_10bit_NR = np.round(full2legal(BT2020_HLG_from_LogC4),0)
            BT2020_PQ_from_LogC4_10bit_NR = np.round(full2legal(BT2020_PQ_from_LogC4),0)

            # Clipping at Maximum Scene Linear Light for each OETF
            BT2020_HLG_from_LogC4_10bit_FR_Clipped = np.clip(BT2020_HLG_from_LogC4_10bit_FR,0,1023)
            BT2020_HLG_from_LogC4_10bit_NR_Clipped_w_Headroom = np.clip(BT2020_HLG_from_LogC4_10bit_NR,0,1023)
            BT2020_HLG_from_LogC4_10bit_NR_Clipped_wo_Headroom = np.clip(BT2020_HLG_from_LogC4_10bit_NR,0,940)
            BT2020_PQ_from_LogC4_10bit_FR_Clipped = np.clip(BT2020_PQ_from_LogC4_10bit_FR,0,1023)
            BT2020_PQ_from_LogC4_10bit_NR_Clipped = np.clip(BT2020_PQ_from_LogC4_10bit_NR,0,940)

            # Reduce to Domain 0...1
            BT2020_HLG_from_LogC4_10bit_0_1_FR = np.divide(BT2020_HLG_from_LogC4_10bit_FR_Clipped,1023)
            BT2020_HLG_from_LogC4_10bit_0_1_NR = np.divide(BT2020_HLG_from_LogC4_10bit_NR_Clipped_w_Headroom,1023)
            BT2020_PQ_from_LogC4_10bit_0_1_FR = np.divide(BT2020_PQ_from_LogC4_10bit_FR_Clipped,1023)
            BT2020_PQ_from_LogC4_10bit_0_1_NR = np.divide(BT2020_PQ_from_LogC4_10bit_NR_Clipped,1023)

            print(np.max(BT2020_PQ_from_LogC4_10bit_0_1_NR))

            # Set Slicing
            y_min =int((height/2)-(target_height/2))
            y_max =int((height/2)+(target_height/2))
            x_min =int((width/2)-(target_width/2))
            x_max =int((width/2)+(target_width/2))

            # Store as file
            #c.write_image(BT2020_HLG_from_LogC4_10bit_0_1_FR[x_min:x_max,y_min:y_max,:],new_name_hlg_FR)
            #c.write_image(BT2020_HLG_from_LogC4_10bit_0_1_NR[x_min:x_max,y_min:y_max,:],new_name_hlg_NR)
            #c.write_image(BT2020_PQ_from_LogC4_10bit_0_1_FR[x_min:x_max,y_min:y_max,:],new_name_pq_FR,bit_depth="uint16")
            #c.write_image(BT2020_PQ_from_LogC4_10bit_0_1_NR[x_min:x_max,y_min:y_max,:],new_name_pq_NR)

            # Store image without Scaling
            #c.write_image(BT2020_HLG_from_LogC4_10bit_0_1_FR,new_name_hlg_FR)
            #c.write_image(BT2020_HLG_from_LogC4_10bit_0_1_NR,new_name_hlg_NR)
            #c.write_image(BT2020_PQ_from_LogC4_10bit_0_1_FR,new_name_pq_FR)
            #c.write_image(BT2020_PQ_from_LogC4_10bit_0_1_NR,new_name_pq_NR)

/Users/justus/Library/CloudStorage/OneDrive-Persönlich/Masterarbeit/Analysis_Content_Encoding/XX_WATCHFOLDER_LOGC4_2_PQ_HLG/20230103_Encounters_V1/2_B002C016_220405_B09C/B002C016_220405_B09C.01513213.tif


/Users/justus/.local/lib/python3.9/site-packages/colour/utilities/verbose.py:262: ColourUsageWarning: "OpenImageIO" related API features are not available, switching to "Imageio"!
  warn(*args, **kwargs)
/Users/justus/.local/lib/python3.9/site-packages/colour/utilities/verbose.py:262: ColourUsageWarning: "Recommendation ITU-R BT.2100" "Reference HLG OOTF" uses RGB Luminance in computations and expects a vector input, thus the given input array will be stacked to compose a vector for internal computations but a single component will be output.
  warn(*args, **kwargs)
/Users/justus/.local/lib/python3.9/site-packages/colour/models/rgb/transfer_functions/arib_std_b67.py:115: RuntimeWarning: invalid value encountered in log
  E <= 1, r * gamma_function(E, 0.5, "mirror"), a * np.log(E - b) + c


0.918866080156
/Users/justus/Library/CloudStorage/OneDrive-Persönlich/Masterarbeit/Analysis_Content_Encoding/XX_WATCHFOLDER_LOGC4_2_PQ_HLG/20230103_Encounters_V1/15_A012C061_220410_R088/A012C061_220410_R088.00164860.tif
0.918866080156
/Users/justus/Library/CloudStorage/OneDrive-Persönlich/Masterarbeit/Analysis_Content_Encoding/XX_WATCHFOLDER_LOGC4_2_PQ_HLG/20230103_Encounters_V1/4_F002C012_220412_R07V/F002C012_220412_R07V.01119678.tif
0.918866080156
/Users/justus/Library/CloudStorage/OneDrive-Persönlich/Masterarbeit/Analysis_Content_Encoding/XX_WATCHFOLDER_LOGC4_2_PQ_HLG/20230103_Encounters_V1/3_B005C001_220420_B09C/B005C001_220420_B09C.00051100.tif
0.918866080156
/Users/justus/Library/CloudStorage/OneDrive-Persönlich/Masterarbeit/Analysis_Content_Encoding/XX_WATCHFOLDER_LOGC4_2_PQ_HLG/20230103_Encounters_V1/12_A003C004_220407_R08Y/A003C004_220407_R08Y.01557645.tif
0.848484848485
/Users/justus/Library/CloudStorage/OneDrive-Persönlich/Masterarbeit/Analysis_Content_Encoding/XX_WATCH

KeyboardInterrupt: 

In [6]:
full2legal(0)

64

### Without Scaling to UHD-1

In [ ]:
# LogC4 to PQ/HLG Conversion with Color Gamut Mapping (Conversion Matrix + Clamping)

root = '/Users/justus/Library/CloudStorage/OneDrive-Persönlich/Masterarbeit/Analysis_Content_Encoding/XX_WATCHFOLDER_LOGC4_2_PQ_HLG'
pattern = "*.tif"

for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            path = os.path.join(path)
            # Folder Settings
            new_path_hlg_FR = path.replace('XX_WATCHFOLDER_LOGC4_2_PQ_HLG',"11.1_TRANSFORMED_HLG_FR")
            new_path_hlg_NR = path.replace('XX_WATCHFOLDER_LOGC4_2_PQ_HLG',"11.2_TRANSFORMED_HLG_NR")
            new_path_pq_FR = path.replace('XX_WATCHFOLDER_LOGC4_2_PQ_HLG',"12.1_TRANSFORMED_PQ_FR_")
            new_path_pq_NR = path.replace('XX_WATCHFOLDER_LOGC4_2_PQ_HLG',"12.2_TRANSFORMED_PQ_NR_")
            # File Settings
            new_name_hlg_FR = os.path.join(new_path_hlg_FR,"TRANSFORMED_HLG_BT2020_FR_" + name)
            new_name_hlg_NR = os.path.join(new_path_hlg_NR,"TRANSFORMED_HLG_BT2020_NR_" + name)
            new_name_pq_FR = os.path.join(new_path_pq_FR,"TRANSFORMED_PQ_BT2020_FR_" + name)
            new_name_pq_NR = os.path.join(new_path_pq_NR,"TRANSFORMED_PQ_BT2020_NR_" + name)
            # make directories
            if not os.path.exists(new_path_hlg_FR):
                os.makedirs(new_path_hlg_FR)
            if not os.path.exists(new_path_hlg_NR):
                os.makedirs(new_path_hlg_NR)
            if not os.path.exists(new_path_pq_FR):
                os.makedirs(new_path_pq_FR)
            if not os.path.exists(new_path_pq_NR):
                os.makedirs(new_path_pq_NR)
            print(os.path.join(path,name))

            LogC4_image = c.read_image(os.path.join(path,name))
            height = len(LogC4_image[0,:,0])
            width = len(LogC4_image[:,0,0])

            target_height = 3840
            target_width = 2160

            LogC4_Linear = Decoding_LogC4(LogC4_image)
            Linear_XYZ = np.matmul(LogC4_Linear,Matrix_LogC4_XYZ)
            Linear_BT2020 = np.matmul(Linear_XYZ,Matrix_XYZ_BT2020)

            # Scaling Scene Linear Reflectance @ 203 Nits HDR Diffuse White
            Scaled_Linear_BT2020_HLG = Linear_BT2020 * c.models.oetf_inverse_HLG_BT2100(c.models.eotf_inverse_HLG_BT2100(203,0,1000))
            Scaled_Linear_BT2020_PQ = Linear_BT2020 * c.models.oetf_inverse_PQ_BT2100(c.models.eotf_inverse_ST2084(203))

            # Converting to Relative Video Level
            BT2020_HLG_from_LogC4 = c.models.oetf_HLG_BT2100(Scaled_Linear_BT2020_HLG)
            BT2020_PQ_from_LogC4 = c.models.oetf_PQ_BT2100(Scaled_Linear_BT2020_PQ)

            # Quantization FR
            BT2020_HLG_from_LogC4_10bit_FR = np.round(np.multiply(BT2020_HLG_from_LogC4,1023),0)
            BT2020_PQ_from_LogC4_10bit_FR = np.round(np.multiply(BT2020_PQ_from_LogC4,1023),0)

            # Quantization NR
            BT2020_HLG_from_LogC4_10bit_NR = np.round(full2legal(BT2020_HLG_from_LogC4),0)
            BT2020_PQ_from_LogC4_10bit_NR = np.round(full2legal(BT2020_PQ_from_LogC4),0)

            # Clipping at Maximum Scene Linear Light for each OETF
            BT2020_HLG_from_LogC4_10bit_FR_Clipped = np.clip(BT2020_HLG_from_LogC4_10bit_FR,0,1023)
            BT2020_HLG_from_LogC4_10bit_NR_Clipped_w_Headroom = np.clip(BT2020_HLG_from_LogC4_10bit_NR,0,1023)
            BT2020_HLG_from_LogC4_10bit_NR_Clipped_wo_Headroom = np.clip(BT2020_HLG_from_LogC4_10bit_NR,0,940)
            BT2020_PQ_from_LogC4_10bit_FR_Clipped = np.clip(BT2020_PQ_from_LogC4_10bit_FR,0,1023)
            BT2020_PQ_from_LogC4_10bit_NR_Clipped = np.clip(BT2020_PQ_from_LogC4_10bit_NR,0,940)

            # Reduce to Domain 0...1
            BT2020_HLG_from_LogC4_10bit_0_1_FR = np.divide(BT2020_HLG_from_LogC4_10bit_FR_Clipped,1023)
            BT2020_HLG_from_LogC4_10bit_0_1_NR = np.divide(BT2020_HLG_from_LogC4_10bit_NR_Clipped_w_Headroom,1023)
            BT2020_PQ_from_LogC4_10bit_0_1_FR = np.divide(BT2020_PQ_from_LogC4_10bit_FR_Clipped,1023)
            BT2020_PQ_from_LogC4_10bit_0_1_NR = np.divide(BT2020_PQ_from_LogC4_10bit_NR_Clipped,1023)

            # Set Slicing
            y_min =int((height/2)-(target_height/2))
            y_max =int((height/2)+(target_height/2))
            x_min =int((width/2)-(target_width/2))
            x_max =int((width/2)+(target_width/2))

            # Store as file
            # c.write_image(BT2020_HLG_from_LogC4_10bit_0_1_FR[x_min:x_max,y_min:y_max,:],new_name_hlg_FR)
            # c.write_image(BT2020_HLG_from_LogC4_10bit_0_1_NR[x_min:x_max,y_min:y_max,:],new_name_hlg_NR)
            # c.write_image(BT2020_PQ_from_LogC4_10bit_0_1_FR[x_min:x_max,y_min:y_max,:],new_name_pq_FR,bit_depth="uint16")
            # c.write_image(BT2020_PQ_from_LogC4_10bit_0_1_NR[x_min:x_max,y_min:y_max,:],new_name_pq_NR)

            # Store image without Scaling
            c.write_image(BT2020_HLG_from_LogC4_10bit_0_1_FR,new_name_hlg_FR)
            c.write_image(BT2020_HLG_from_LogC4_10bit_0_1_NR,new_name_hlg_NR)
            c.write_image(BT2020_PQ_from_LogC4_10bit_0_1_FR,new_name_pq_FR)
            c.write_image(BT2020_PQ_from_LogC4_10bit_0_1_NR,new_name_pq_NR)